In [13]:
import numpy as np
import pandas as pd
import risk_kit_nishtha as rk
from datetime import datetime
import math

In [10]:
dateparse = lambda x: datetime.strptime(x, '%Y%m')

df = pd.read_csv("ind30_m_vw_rets.csv", parse_dates=['Unnamed: 0'], date_parser=dateparse)
df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df.set_index('Date', inplace=True)
df

C:\Users\govil\AppData\Local\Temp\ipykernel_263376\302198592.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv("ind30_m_vw_rets.csv", parse_dates=['Unnamed: 0'], date_parser=dateparse)


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm,Servs,BusEq,Paper,Trans,Whlsl,Rtail,Meals,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,0.56,-5.19,1.29,2.93,10.97,-0.48,8.08,1.77,8.14,0.39,...,0.83,9.22,2.06,7.70,1.93,-23.79,0.07,1.87,0.37,5.20
1926-08-01,2.59,27.03,6.50,0.55,10.01,-3.58,-2.51,4.25,5.50,8.14,...,2.17,2.02,4.39,-2.38,4.88,5.39,-0.75,-0.13,4.46,6.76
1926-09-01,1.16,4.02,1.26,6.58,-0.99,0.73,-0.51,0.69,5.33,2.31,...,2.41,2.25,0.19,-5.54,0.05,-7.87,0.25,-0.56,-1.23,-3.86
1926-10-01,-3.06,-3.31,1.06,-4.76,9.47,-4.68,0.12,-0.57,-4.76,1.00,...,-0.11,-2.00,-1.09,-5.08,-2.64,-15.38,-2.20,-4.11,-5.16,-8.49
1926-11-01,6.35,7.29,4.55,1.66,-5.80,-0.54,1.87,5.42,5.20,3.11,...,1.63,3.77,3.64,3.84,1.60,4.67,6.52,4.33,2.24,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-01,-0.38,-1.86,-5.43,2.89,-4.47,2.54,5.26,4.46,0.01,3.80,...,2.95,5.24,9.93,-0.34,2.91,3.66,9.11,3.64,2.45,2.99
2018-09-01,-0.32,0.19,5.37,-0.09,-2.21,1.07,1.30,1.99,-2.87,-6.38,...,1.74,-0.37,-0.33,-0.30,1.05,-1.48,0.61,2.51,-1.93,1.16
2018-10-01,1.02,-1.57,7.90,-15.96,-6.66,-0.51,-10.14,-8.84,-12.50,-25.79,...,-0.50,-9.20,-8.06,-9.82,-9.75,-7.88,-10.21,-1.71,-5.45,-5.99


In [17]:
df_col = df.columns.values.tolist()
SR_dict = {}
for i in df_col:
    sr = -rk.sharpe_ratio(df[i])
    SR_dict[i] = sr

SR_dict

TypeError: unsupported operand type(s) for /: 'float' and 'function'

In [ ]:
Risky_assets = ['Food','Hshld','Util','Telcm']
Safe_assets = ['Beer','Smoke','Txtls','Coal','Servs']

In [ ]:
#Creating dataframe of risky and safe assets
df_risky = pd.DataFrame()
df_safe = pd.DataFrame()
for i in Risky_assets:
    df_risky[i] = df[i]['2000-01-01':]
for j in Safe_assets:
    df_safe[j] = df[j]['2000-01-01':]

In [ ]:
#Calculate returns of risky asset
n = len(Risky_assets)
weights_r = np.full(n,1/n).reshape(-1,1)
Risky_r = rk.portfolio_return(df_risky,weights_r)

#Calculate returns of safe asset
m = len(Safe_assets)
weights_s = np.full(m,1/m).reshape(-1,1)
Safe_r = rk.portfolio_return(df_safe,weights_s)

Risky_r = pd.DataFrame(Risky_r, index=df_risky.index)
Safe_r = pd.DataFrame(Safe_r, index=df_safe.index)

In [ ]:
def cppi(risky_assets, safe_assets, multiplier, start_value,floor_value):
    
    # Creating parameters for the strategy
    dates = risky_assets.index
    n_steps = len(dates)
    account_value = start_value
    floor_value = start_value*floor
    
    # Creating dataframes to store values
    portfolio_values = pd.DataFrame(index = risky_assets.index)
    cushion_values = pd.DataFrame(index = risky_assets.index)
    risky_weights = pd.DataFrame(index = risky_assets.index)
    peaks = pd.DataFrame(index = risky_assets.index)
    
    for step in range(n_steps):
        cushion = (account_value-floor_value)/account_value
        risky_w = multiplier*cushion
        risky_w = min(risky_w,1)
        risky_w = max(risky_w,0)
        safe_w = 1-risky_w
    
        risky_alloc = risky_w*account_value
        safe_alloc = safe_w*account_value
        
        account_value = risky_alloc*(1+risky_assets.iloc[step]) + safe_alloc*(1+safe_assets.iloc[step])
        
        peak_value = np.max(portfolio_values)
        
        portfolio_values.iloc[step] = account_value
        cushion_values.iloc[step] = cushion
        risky_weights.iloc[step] = risky_w
        peaks.iloc[step] = peak_value
        
    risky_values = start_value*(1+risky_assets).cumprod()
    prev_peak = risky_values.cummax()
    drawdown = (risky_values-prev_peak)/prev_peaks
    
    if drawdown is not None:
        peaks = np.max(peaks,portfolio_values)
        floor_level = peak*(1-drawdown)
        
    # Creating empty DataFrame for storing final results
    results = pd.DataFrame(index = risky_assets.index)
    
    # Store the results in the DataFrame
    results['Account Value'] = portfolio_values
    results['Risky Value'] = risky_values
    results['Cushion History'] = cushion_values
    results['Risky Weight'] = risky_weights
    results['Multiplier'] = multiplier
    results['Drawdown'] = drawdown
    results['Peak'] = peaks
    results['Floor History'] = floor_level
    results['Risky Returns'] = risky_assets
    results['Safe Returns'] = safe_assets
    
    return results